# Revised Prompt/Recursive Generations Notebook

This notebook is made up of several functions that:

* *generates an image* from a prompt
  * while exporting the "revised prompt," created by OpenAI from your "original prompt," that was used to create the image
* uses this revised prompt to generate a new image (in this case "the opposite")
* then loops through this several times



## TO DOs

to make images from prompts and save those images + prompts, you'll need to do two things:



1.   Set your API key
2.   Set the "output" paths to your own gDrive/preferred folders.
3.   Put in your prompt.



#### API KEY

In the code, you'll see that we have the key referenced as OPENAI_API_KEY. This is what you'll need to name your key as you input it into your "secrets" tab. Gif below to demonstrate the location of all these elements.

![alt text](https://files.slack.com/files-pri/T0HTW3H0V-F06NG72EQ5U/screen_recording_mar_8.gif?pub_secret=12dc4ca9c9)




In [ ]:
!pip install openai pillow
import base64, io, textwrap, os, datetime
from PIL import Image
from IPython.display import display, Markdown
from pathlib import Path


In [ ]:
from google.colab import userdata, drive
OPENAI_API_KEY = userdata.get("OPENAI_API_KEY")
drive.mount("/content/drive")

from openai import OpenAI
client = OpenAI(api_key=OPENAI_API_KEY)


In [ ]:
initial_prompt = "Create a visual interpretation of Jorge Luis Borges’ poem 'Otro poema de los dones', designed for an English-speaking audience, using symbolic imagery that reflects its themes of gratitude, libraries, and the infinite."
num_loops      = 5

img_model      = "dall-e-3"
opposite_model = "gpt-4o"
img_size       = "1024x1024"
wrap_width     = 80

In [ ]:
prompt_history   = []
image_b64_list   = []

current_prompt = initial_prompt

for i in range(num_loops):
    img_resp = client.images.generate(
        model=img_model,
        prompt=current_prompt,
        n=1,
        size=img_size,
        response_format="b64_json",
    )
    img_data  = img_resp.data[0]
    revised   = img_data.revised_prompt or current_prompt
    img_b64   = img_data.b64_json

    prompt_history.append(revised)
    image_b64_list.append(img_b64)

    img = Image.open(io.BytesIO(base64.b64decode(img_b64)))
    display(img)
    print("-"*90)
    print(textwrap.fill(f"[{i+1}] {revised}", width=wrap_width))
    print("-"*90)

    opp_resp = client.responses.create(
        model=opposite_model,
        input=[
            {
                "role": "system",
                "content": (
                    "Provide the opposite of the user's image prompt. "
                    "Return ONLY that opposite prompt—no extra words."
                ),
            },
            {"role": "user", "content": revised},
        ],
    )
    current_prompt = opp_resp.output_text.strip()

print("\nNext opposite prompt would be:\n",
      textwrap.fill(current_prompt, width=wrap_width))


## outputting to gdrive folders:

In [ ]:
img_dir = Path("/content/drive/Shareddrives/LL_CENTRAL/LL_PROJECTS/24.1/COLAB/ll_color_colabs/_ai_lab/utilities/output-folder/images")
txt_dir = Path("/content/drive/Shareddrives/LL_CENTRAL/LL_PROJECTS/24.1/COLAB/ll_color_colabs/_ai_lab/utilities/output-folder/text")
wrap_w  = 80

img_dir.mkdir(parents=True, exist_ok=True)
txt_dir.mkdir(parents=True, exist_ok=True)

timestamp_fmt = "%Y%m%d-%H%M%S"

for idx, (prompt, img_b64) in enumerate(zip(prompt_history, image_b64_list), start=1):
    stamp = datetime.datetime.now().strftime(timestamp_fmt)
    stem  = f"{idx:02d}_{stamp}"

    # save image
    (img_dir / f"{stem}.png").write_bytes(base64.b64decode(img_b64))

    # save wrapped prompt
    wrapped = textwrap.fill(prompt, width=wrap_w)
    (txt_dir / f"{stem}.txt").write_text(wrapped, encoding="utf-8")

print(f"Saved {len(prompt_history)} image / prompt pairs →")
print(f"    • {img_dir}\n    • {txt_dir}")
